-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Writing to Delta Tables
**Delta Lake tables provide ACID compliant updates to tables backed by data files in cloud object storage**.

In this notebook, we'll explore SQL syntax to process updates with Delta Lake. While many operations are standard SQL, slight variations exist to accommodate Spark and Delta Lake execution.

## Learning Objectives
By the end of this lesson, you should be able to:
- Overwrite data tables using **`INSERT OVERWRITE`**
- Append to a table using **`INSERT INTO`**
- Append, update, and delete from a table using **`MERGE INTO`**
- Ingest data incrementally into tables using **`COPY INTO`**

## Run Setup

The setup script will create the data and declare necessary values for the rest of this notebook to execute.

In [0]:
%run ../Includes/Classroom-Setup-4.4

Python interpreter will be restarted.
Python interpreter will be restarted.

Dropping the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_4_4"
Removing the working directory "dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/4.4"

Creating the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_4_4"
Skipping install to "dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss", dataset already exists

Cloning the sales table from dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss/delta/sales_hist...(4 seconds / 10,510 records)
Cloning the users table from dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss/delta/users_hist...(2 seconds / 251,501 records)
Cloning the events table from dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss/delta/events_hist...(3 seconds / 485,696 records)
Cloning the users_update table from dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss/delta/users_update...(3 seconds / 917 records)
Cloning the events_update table from dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss/delta/events_update...(4 seconds / 1,927 records)

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/4.4
 DA.paths.user_db: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/4.4/4_4.db
 DA.paths.datasets: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss

Predefined tables in dbacademy_manujkumar_joshi_celebaltech_com_dewd_4_4:
 events
 events_update
 sales
 users
 users_update

Setup completed in 20 seconds

## Complete Overwrites

**We can use overwrites to atomically replace all of the data in a table. There are multiple benefits to overwriting tables instead of deleting and recreating tables**:
- Overwriting a table is much faster because it doesn’t need to list the directory recursively or delete any files.
- The old version of the table still exists; can easily retrieve the old data using Time Travel.
- It’s an atomic operation. Concurrent queries can still read the table while you are deleting the table.
- Due to ACID transaction guarantees, if overwriting the table fails, the table will be in its previous state.

#### Spark SQL provides two easy methods to accomplish complete overwrites.

Some students may have noticed previous lesson on CTAS statements actually used CRAS statements (to avoid potential errors if a cell was run multiple times).

## 1. **`CREATE OR REPLACE TABLE`** (CRAS) statements fully replace the contents of a table each time they execute.

In [0]:
%sql
CREATE OR REPLACE TABLE events AS
SELECT * FROM parquet.`${da.paths.datasets}/raw/events-historical`

num_affected_rows,num_inserted_rows


Reviewing the table history shows a previous version of this table was replaced.

In [0]:
%sql
DESCRIBE HISTORY events

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2022-07-29T06:23:35.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2331746562401965),0725-045645-b5m629fz,0,WriteSerializable,false,"Map(numFiles -> 4, numOutputRows -> 485696, numOutputBytes -> 15283922)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-07-29T06:21:19.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,CLONE,"Map(source -> delta.`dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss/delta/events_hist`, sourceVersion -> 1, isShallow -> true)",null,List(2331746562401965),0725-045645-b5m629fz,-1,Serializable,false,"Map(removedFilesSize -> 0, numRemovedFiles -> 0, sourceTableSize -> 14998176, numCopiedFiles -> 0, copiedFilesSize -> 0, sourceNumOfFiles -> 1)",null,Databricks-Runtime/10.4.x-scala2.12


## 2. `INSERT OVERWRITE` provides a nearly identical outcome as above: data in the target table will be replaced by data from the query. 

**`INSERT OVERWRITE`**:

- **Can only overwrite an existing table, not create a new one like our CRAS statement**
- Can overwrite only with new records that match the current table schema -- and thus can be a "safer" technique for overwriting an existing table without disrupting downstream consumers
- Can overwrite individual partitions

In [0]:
%sql
INSERT OVERWRITE sales
SELECT * FROM parquet.`${da.paths.datasets}/raw/sales-historical/`

num_affected_rows,num_inserted_rows
10510,10510


Note that different metrics are displayed than a CRAS statement; the table history also records the operation differently.

In [0]:
%sql
DESCRIBE HISTORY sales

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2022-07-29T06:25:39.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(2331746562401965),0725-045645-b5m629fz,0,WriteSerializable,false,"Map(numFiles -> 4, numOutputRows -> 10510, numOutputBytes -> 353497)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-07-29T06:21:13.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,CLONE,"Map(source -> delta.`dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss/delta/sales_hist`, sourceVersion -> 1, isShallow -> true)",null,List(2331746562401965),0725-045645-b5m629fz,-1,Serializable,false,"Map(removedFilesSize -> 0, numRemovedFiles -> 0, sourceTableSize -> 334175, numCopiedFiles -> 0, copiedFilesSize -> 0, sourceNumOfFiles -> 1)",null,Databricks-Runtime/10.4.x-scala2.12


A primary difference here has to do with how Delta Lake enforces schema on write.

#### Whereas a CRAS statement will allow us to completely redefine the contents of our target table, **`INSERT OVERWRITE`** will fail if we try to change our schema (unless we provide optional settings). 

Uncomment and run the cell below to generated an expected error message.

In [0]:
%sql
-- INSERT OVERWRITE sales
-- SELECT *, current_timestamp() FROM parquet.`${da.paths.datasets}/raw/sales-historical`

## 3. Append Rows

We can use **`INSERT INTO`** to atomically append new rows to an existing Delta table. This allows for incremental updates to existing tables, which is much more efficient than overwriting each time.

Append new sale records to the **`sales`** table using **`INSERT INTO`**.

In [0]:
%sql
INSERT INTO sales
SELECT * FROM parquet.`${da.paths.datasets}/raw/sales-30m`

num_affected_rows,num_inserted_rows
29,29


Note that **`INSERT INTO`** does not have any built-in guarantees to prevent inserting the same records multiple times. Re-executing the above cell would write the same records to the target table, resulting in duplicate records.

## 4. Merge Updates

**You can upsert data from a source table, view, or DataFrame into a target Delta table using the **`MERGE`** SQL operation**. Delta Lake supports inserts, updates and deletes in **`MERGE`**, and supports extended syntax beyond the SQL standards to facilitate advanced use cases.

<strong><code>
MERGE INTO target a<br/>
USING source b<br/>
ON {merge_condition}<br/>
WHEN MATCHED THEN {matched_action}<br/>
WHEN NOT MATCHED THEN {not_matched_action}<br/>
</code></strong>

We will use the **`MERGE`** operation to update historic users data with updated emails and new users.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW users_update AS 
SELECT *, current_timestamp() AS updated 
FROM parquet.`${da.paths.datasets}/raw/users-30m`

The main benefits of **`MERGE`**:
* **updates, inserts, and deletes are completed as a single transaction**
* multiple conditionals can be added in addition to matching fields
* provides extensive options for implementing custom logic

Below, we'll only update records if the current row has a **`NULL`** email and the new row does not. 

All unmatched records from the new batch will be inserted.

In [0]:
%sql
MERGE INTO users a
USING users_update b
ON a.user_id = b.user_id
WHEN MATCHED AND a.email IS NULL AND b.email IS NOT NULL THEN
  UPDATE SET email = b.email, updated = b.updated
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
983,72,0,911


Note that we explicitly specify the behavior of this function for both the **`MATCHED`** and **`NOT MATCHED`** conditions; the example demonstrated here is just an example of logic that can be applied, rather than indicative of all **`MERGE`** behavior.

## 5. Insert-Only Merge for Deduplication

A common ETL use case is to collect logs or other every-appending datasets into a Delta table through a series of append operations. 

Many source systems can generate duplicate records. With merge, you can avoid inserting the duplicate records by performing an insert-only merge.

This optimized command uses the same **`MERGE`** syntax but only provided a **`WHEN NOT MATCHED`** clause.

Below, we use this to confirm that records with the same **`user_id`** and **`event_timestamp`** aren't already in the **`events`** table.

In [0]:
%sql
MERGE INTO events a
USING events_update b
ON a.user_id = b.user_id AND a.event_timestamp = b.event_timestamp
WHEN NOT MATCHED AND b.traffic_source = 'email' THEN 
  INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
128,0,0,128


## 6. Load Incrementally

**`COPY INTO`** provides SQL engineers an idempotent option to incrementally ingest data from external systems.

Note that this operation does have some expectations:
- Data schema should be consistent
- Duplicate records should try to be excluded or handled downstream

This operation is potentially much cheaper than full table scans for data that grows predictably.

While here we'll show simple execution on a static directory, the real value is in multiple executions over time picking up new files in the source automatically.

In [0]:
%sql
COPY INTO sales
FROM "${da.paths.datasets}/raw/sales-30m"
FILEFORMAT = PARQUET

num_affected_rows,num_inserted_rows
29,29


Run the following cell to delete the tables and files associated with this lesson.

In [0]:
%python
DA.cleanup()

Dropping the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_4_4"
Removing the working directory "dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/4.4"

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>